In [ ]:
!nvidia-smi

In [ ]:
!pip install bitsandbytes

In [ ]:
!pip install bitsandbytes
!pip install accelerate
!pip install --upgrade transformers
!pip install --upgrade peft
!pip install --upgrade datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

Load model and tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T", padding_side="right",)
tokenizer.pad_token = tokenizer.eos_token
bnb_config = BitsAndBytesConfig(
   load_in_8bit=True,
#    bnb_4bit_quant_type="nf4",
#    bnb_4bit_use_double_quant=True,
   bnb_8bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T", device_map="auto", quantization_config=bnb_config)

In [ ]:
txt = """###SYSTEM: Based on INPUT title generate the prompt for generative model

###INPUT: Linux Terminal

###PROMPT:"""
tokens = tokenizer(txt, return_tensors="pt")['input_ids'].to("cuda")
op = model.generate(tokens, max_new_tokens=200)
print(tokenizer.decode(op[0]))

Preparing PEFT model

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1, peft_type=TaskType.CAUSAL_LM)
model_lora = get_peft_model(model, peft_config)

print(model_lora.print_trainable_parameters())

Prepare dataset

In [ ]:
def format_dataset(data_point):
    prompt = f"""###SYSTEM: Based on INPUT title generate the prompt for generative model

###INPUT: {data_point['act']}

###PROMPT: {data_point['prompt']}
"""if 
    tokens = tokenizer(prompt,
        truncation=True,
        max_length=256,
        padding="max_length",)
    tokens["labels"] = tokens['input_ids'].copy()
    return tokens

In [45]:
import json, re
from pathlib import Path

In [ ]:
print(tokenizer.decode(dataset[0]['input_ids']))

In [47]:
from datasets import load_dataset

dataset = load_dataset("fka/awesome-chatgpt-prompts", split="train")
print(dataset[0].keys())

dataset = dataset.map(format_dataset)
print(dataset[0].keys())

Parsed examples: 1
{'system': '[SYSTEM]: This is a task you must complete by returning only the output.\nDo not include explanations, code, or extra text—only the result.\n', 'user': '[USER]: create_a_student\n\npromp = I am John', 'assistant': '[ASSISTANT]: {\n"name": "John"\n}'}


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [53]:
print(ds_full)

Dataset({
    features: ['system', 'user', 'assistant', 'text'],
    num_rows: 1
})


In [52]:
print(ds_full["text"])

Column(['###SYSTEM: [SYSTEM]: This is a task you must complete by returning only the output.\nDo not include explanations, code, or extra text—only the result.\n\n###INPUT: [USER]: create_a_student\n\npromp = I am John\n\n###PROMPT: [ASSISTANT]: {\n"name": "John"\n}'])


In [50]:

print(dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 203
})


In [ ]:
tmp = dataset.train_test_split(test_size=0.1)
train_dataset = tmp["train"]
test_dataset = tmp["test"]
print(train_dataset)
print(test_dataset)

In [ ]:
import torch
if torch.cuda.device_count() > 1:
    model_lora.is_parallelizable = True
    model_lora.model_parallel = True

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
                    model = model_lora,
                    train_dataset=train_dataset,
                    eval_dataset = test_dataset,
                    tokenizer = tokenizer,
                    data_collator = data_collator,

                    args = TrainingArguments(
                        output_dir="./training",
                        remove_unused_columns=False,
                        per_device_train_batch_size=2,
                        gradient_checkpointing=True,
                        gradient_accumulation_steps=4,
                        max_steps=200,
                        learning_rate=2.5e-5,
                        logging_steps=5,
                        fp16=True,
                        optim="paged_adamw_8bit",
                        save_strategy="steps",
                        save_steps=50,
                        eval_strategy="steps",
                        eval_steps=5,
                        do_eval=True,
                        label_names = ["input_ids", "labels", "attention_mask"],
                        report_to = "none",

                ))

In [ ]:
trainer.train()

Generating text

In [ ]:
txt = """###SYSTEM: Based on INPUT title generate the prompt for generative model

###INPUT: Pirate

###PROMPT:"""
tokens = tokenizer(txt, return_tensors="pt")['input_ids'].to("cuda")
op = model_lora.generate(tokens, max_new_tokens=200)
print(tokenizer.decode(op[0]))

Saving PEFT model lora

In [ ]:
model.save_pretrained("prompt_250_steps", safe_serialization=False, )

In [ ]:
!zip -r prompt_250.zip '/kaggle/working/prompt_250_steps'

Loading PEFT model weights

In [ ]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, "/kaggle/working/prompt_250_steps")

Merging PEFT Weights into Base model and saving

In [ ]:
model_ = model.merge_and_unload()
model_.save_pretrained("merged_model_")